#### Cleaning Fitbit Sleep

Goal:
- Produce a reliable Fitbit-only user–day sleep table
- Preserve missingness
- Flag invalid values instead of forcing corrections
- Do not infer or fill using other devices

Identity:
- Drop rows where Id is missing or blank, as user-level analysis is not possible.

Sleep timestamps:
- Parse sleep_start and sleep_end into datetime objects.
- Calculate sleep duration from timestamps for validation.
- Flag segments with implausibly long durations.

Segment selection:
- Normalize isMainSleep into a boolean.
- Use main sleep segments when available.
- If no main sleep segment exists for a night, fall back to all segments.

Date assignment:
- Assign sleep to the date of sleep_end (wake-up date).

Aggregation:
- Aggregate multiple segments per user–day by summing minutesAsleep and minutesAwake.
- Average sleep_efficiency across segments.
- Flag nights with multiple segments.

Sleep efficiency:
- Keep as-is; allow missing values.

Columns:
- Retain all metric and metadata columns at this stage.
- Do not drop columns unless unusable for analysis.

In [1]:
import pandas as pd
import duckdb

In [2]:
fitbit_sleep = pd.read_csv("wearables_synthetic_raw/fitbit_sleep_2024.csv")

In [3]:
con = duckdb.connect()

In [4]:
con.register("fitbit_sleep", fitbit_sleep)

In [5]:
con.execute("SELECT COUNT(*) FROM fitbit_sleep").fetchall()

[(25264,)]

In [6]:
#bad Ids
con.execute("""SELECT COUNT(*) AS total_rows,
SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END) AS null_id_rows,
SUM(CASE WHEN id IS NOT NULL AND TRIM(CAST(id AS VARCHAR)) = '' THEN 1 ELSE 0 END)
AS blank_id_rows
FROM fitbit_sleep""").df()

,total_rows,null_id_rows,blank_id_rows
0,25264,0.0,0.0


In [7]:
#identity rule
con.execute("""CREATE OR REPLACE VIEW fitbit_sleep_clean_step1 AS
SELECT * FROM fitbit_sleep
WHERE id IS NOT NULL AND TRIM(CAST(id AS VARCHAR)) <> ''""")

In [8]:
con.execute("""SELECT COUNT(*) AS clean_rows
FROM fitbit_sleep_clean_step1""").df()

,clean_rows
0,25264


0 missing/blank ids so no rows dropped.

In [9]:
con.execute("""SELECT sleep_start, sleep_end FROM fitbit_sleep_clean_step1
LIMIT 12""").df()

,sleep_start,sleep_end
0,2024/01/01 20:51,2024-01-01 23:44:00
1,2024-01-01 22:25:00,2024-01-02 01:20:00
2,2024-01-02 22:17:00,2024-01-03 04:17:00
3,2024-01-03 23:24:00,2024-01-04 05:24:00
4,2024-01-04 22:51:00,2024-01-05 01:43:00
5,2024/01/04 21:31,2024-01-04 23:32:00
6,2024-01-04 20:46:00,2024-01-04 22:55:00
7,2024-01-05 22:22:00,2024-01-06 04:22:00
8,2024/01/06 21:37,2024-01-07 03:37:00
9,2024-01-07 21:09:00,2024-01-07 23:16:00


In [10]:
con.execute("""SELECT 
SUM(CASE WHEN sleep_start LIKE '%/%' THEN 1 ELSE 0 END) AS start_with_slash,
SUM(CASE WHEN sleep_start LIKE '%-%' THEN 1 ELSE 0 END) AS start_with_dash
FROM fitbit_sleep_clean_step1""").df()

,start_with_slash,start_with_dash
0,7573.0,17691.0


In [11]:
con.execute("""CREATE OR REPLACE VIEW fitbit_sleep_clean_step2 AS
SELECT *,
COALESCE(
try_strptime(sleep_start, '%Y-%m-%d %H:%M:%S'),
try_strptime(sleep_start, '%Y/%m/%d %H:%M')) AS sleep_start_ts,
COALESCE(
try_strptime(sleep_end, '%Y-%m-%d %H:%M:%S'),
try_strptime(sleep_end, '%Y/%m/%d %H:%M')) AS sleep_end_ts
FROM fitbit_sleep_clean_step1""")

In [12]:
con.execute("""SELECT COUNT(*) AS rows,
SUM(CASE WHEN sleep_start_ts IS NULL THEN 1 ELSE 0 END) AS unparsed_start,
SUM(CASE WHEN sleep_end_ts IS NULL THEN 1 ELSE 0 END) AS unparsed_end
FROM fitbit_sleep_clean_step2""").df()

,rows,unparsed_start,unparsed_end
0,25264,0.0,0.0


In [13]:
#normalize isMainSleep
con.execute("""SELECT isMainSleep, COUNT(*) AS n FROM fitbit_sleep_clean_step2
GROUP BY isMainSleep
ORDER BY n DESC""").df()

,isMainSleep,n
0,None,3722
1,TRUE,3670
2,1,3669
3,FALSE,3613
4,False,3567
5,0,3536
6,True,3487


In [14]:
con.execute("""CREATE OR REPLACE VIEW fitbit_sleep_clean_step3 AS
SELECT *,
CASE
WHEN lower(trim(CAST(isMainSleep AS VARCHAR))) IN ('true', '1') THEN 1
WHEN lower(trim(CAST(isMainSleep AS VARCHAR))) IN ('false', '0') THEN 0
ELSE NULL
END AS is_main_sleep,
CASE
WHEN lower(trim(CAST(isMainSleep AS VARCHAR))) IN ('true','1','false','0') THEN 0 
ELSE 1
END AS main_sleep_unknown_flag
FROM fitbit_sleep_clean_step2""")

In [15]:
con.execute("""SELECT COUNT(*) AS rows,
SUM(CASE WHEN is_main_sleep = 1 THEN 1 ELSE 0 END) AS main_sleep_rows,
SUM(CASE WHEN is_main_sleep = 0 THEN 1 ELSE 0 END) AS non_main_sleep_rows,
SUM(main_sleep_unknown_flag) AS unknown_rows
FROM fitbit_sleep_clean_step3""").df()

,rows,main_sleep_rows,non_main_sleep_rows,unknown_rows
0,25264,10826.0,10716.0,3722.0


Normalized isMainSleep into boolean

~15% of segments have unknown main/non-main status and are retained for fallback aggregation

In [16]:
#sleep date
con.execute("""CREATE OR REPLACE VIEW fitbit_sleep_clean_step4 AS 
SELECT *,
CAST(sleep_end_ts AS DATE) AS sleep_date,
date_diff('minute', sleep_start_ts, sleep_end_ts) AS segment_minutes
FROM fitbit_sleep_clean_step3""")

In [17]:
con.execute("""SELECT
MIN(segment_minutes) AS min_minutes,
MAX(segment_minutes) AS max_minutes,
approx_quantile(segment_minutes, 0.5) AS p50_minutes,
approx_quantile(segment_minutes, 0.95) AS p95_minutes,
approx_quantile(segment_minutes, 0.99) AS p99_minutes
FROM fitbit_sleep_clean_step4""").df()

,min_minutes,max_minutes,p50_minutes,p95_minutes,p99_minutes
0,40,360,192,360,360


In [18]:
con.execute("""CREATE OR REPLACE VIEW fitbit_sleep_clean_step5 AS
SELECT *,
CASE 
WHEN segment_minutes <=0 THEN 1
WHEN segment_minutes >720 THEN 1
ELSE 0
END AS segment_duration_flag
FROM fitbit_sleep_clean_step4""")

In [19]:
con.execute("""SELECT COUNT(*) AS rows,
SUM(segment_duration_flag) AS flagged_segments
FROM fitbit_sleep_clean_step5""").df()

,rows,flagged_segments
0,25264,0.0


- duration logic is correct
- no segments need removal - all segments retained
- clean flag column

In [20]:
#Aggregae Sleep Segments -> User-Day Sleep
con.execute("""CREATE OR REPLACE VIEW fitbit_sleep_has_main AS 
SELECT id, sleep_date, 
MAX(CASE WHEN is_main_sleep = 1 THEN 1 ELSE 0 END) AS has_main_sleep
FROM fitbit_sleep_clean_step5
GROUP BY id, sleep_date""")

In [21]:
con.execute("""CREATE OR REPLACE VIEW fitbit_sleep_segments_selected AS
SELECT s.* FROM fitbit_sleep_clean_step5 s
JOIN fitbit_sleep_has_main h
ON s.id = h.id
AND s.sleep_date = h.sleep_date
WHERE (h.has_main_sleep = 1 AND s.is_main_sleep = 1) OR (h.has_main_sleep = 0)""")

In [22]:
con.execute("""CREATE OR REPLACE VIEW fitbit_sleep_daily_clean AS
SELECT
id AS user_id,
sleep_date,
SUM(minutesAsleep) AS total_minutes_asleep,
SUM(minutesAwake) AS total_minutes_awake,
AVG(sleep_efficiency) AS avg_sleep_efficiency,
COUNT(*) AS segment_count,
MAX(main_sleep_unknown_flag) AS has_unknown_main_sleep
FROM fitbit_sleep_segments_selected
GROUP BY id, sleep_date""")

In [23]:
con.execute("""SELECT
COUNT(*) AS rows,
AVG(total_minutes_asleep) AS avg_sleep_minutes,
MAX(segment_count) AS max_segments_per_night
FROM fitbit_sleep_daily_clean""").df()

,rows,avg_sleep_minutes,max_segments_per_night
0,14557,306.016274,5


- Fitbit sleep segments aggregated to user-day using wakeup-date - yields fewer user-days than activity due to missing nights and fragmented sleep logging.
- Main sleep segments used when available; otherwise all segments retained.
- Sleep duration and efficiency aggregated conservatively with flags preserved.

In [24]:
con.execute("""COPY fitbit_sleep_daily_clean
TO 'data/clean/fitbit_sleep_daily_clean.csv'
(HEADER, DELIMITER ',')""")

Saved cleaned Fitbit sleep daily data to data/clean/fitbit_sleep_daily_clean.csv.